<a href="https://colab.research.google.com/github/ImagingDataCommons/CloudSegmentator/blob/v1.2.0/util/pricingOptimization/Top_20_cheapest_GPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import json
import pandas as pd
import requests

url = 'https://cloudpricingcalculator.appspot.com/static/data/pricelist.json'
response = requests.get(url)
data = response.json()

df = pd.DataFrame.from_dict(data["gcp_price_list"])
df.index.name = 'Region'

df = df.loc[:, df.columns.str.contains("PREEMPTIBLE")]
df = df.loc[:, ~df.columns.str.contains("EXTENDED|CONFIDENTIAL|NETWORK|TPU")]
df.columns = df.columns.str.replace('CP-COMPUTEENGINE-|-PREEMPTIBLE', '', regex=True)

regions= ['asia-east1', 'asia-east2',
          'asia-northeast1', 'asia-northeast2', 'asia-northeast3',
          'asia-south1', 'asia-south2',
          'asia-southeast1', 'asia-southeast2',
          'australia-southeast1', 'australia-southeast2',
          'europe-central2',
          'europe-north1',
          'europe-southwest1',
          'europe-west1', 'europe-west2','europe-west3','europe-west4','europe-west6','europe-west8','europe-west9','europe-west12','europe-west10',
          'northamerica-northeast1','northamerica-northeast2',
          'southamerica-east1','southamerica-west1',
          'us-central1',
          'us-east1','us-east4','us-east5',
          'us-south1',
          'us-west1','us-west2','us-west3', 'us-west4',
          'me-central', 'me-central2','me-west1']

df1 = df[df.index.isin(regions)]
predefined=df1.loc[:, df1.columns.str.contains("PREDEFINED")& ~df1.columns.str.startswith('PREDEFINED')]
predefined.columns = predefined.columns.str.replace('PREDEFINED-VM-', '', regex=True)
predefined.columns = predefined.columns.str.replace('CORE', 'CPU', regex=True)
gpu=df1.loc[:, df1.columns.str.contains("GPU")]
gpu.columns = gpu.columns.str.replace('GPU_NVIDIA_TESLA_', '', regex=True)

new_columns = []

# Iterate over the columns in the DataFrame
for col in predefined.columns:
    # Split the column name into the CPU family and column name
    cpu_family, col_name = col.split('-')
    # Append a tuple with the CPU family and column name to the new_columns list
    new_columns.append((cpu_family, col_name))

# Create a multi-level column index using the new_columns list
predefined.columns = pd.MultiIndex.from_tuples(new_columns)
cpu_count = int(input('Enter the CPU count: '))
ram_value = int(input('Enter the RAM value (GB): '))
hours = int(input('Enter the hours value: '))
gpu_type = str(input('Select a GPU: '))
gpu=gpu[gpu_type]
# Calculate the pricing for each region and CPU family
pricing = predefined.apply(lambda row: (row.xs('CPU', level=1) * cpu_count + row.xs('RAM', level=1) * ram_value)*hours, axis=1)
gpu=gpu.apply(lambda row: row*hours)
total_pricing=pricing.join(gpu)
for col in pricing.columns:
    if col != gpu_type:
        total_pricing[f'{col}'] = total_pricing[col] + total_pricing[gpu_type]

total_pricing.drop(columns=total_pricing.columns[-1], inplace=True)
#sorted_pricing = total_pricing.stack().sort_values()
gpu_total = total_pricing['N1'].sort_values()
# Select the top 10 cheapest pricings
top_10_pricings = gpu_total.head(20)
top_10_pricings

Enter the CPU count: 4
Enter the RAM value (GB): 16
Enter the hours value: 730
Select a GPU: T4


Region
europe-west2          87.513130
asia-northeast1       92.150090
asia-southeast1      105.102480
us-east4             108.787520
us-east5             110.151160
asia-southeast2      118.426440
asia-northeast2      120.298160
europe-west6         122.129000
europe-west9         127.775346
europe-west8         127.775346
us-west4             128.263920
europe-west4         129.799840
us-south1            129.978369
europe-southwest1    129.978369
asia-east1           130.845200
us-central1          132.051160
us-west1             132.051160
us-east1             132.051160
europe-west1         137.584560
asia-northeast3      139.278160
Name: N1, dtype: float64

In [3]:
sorted_pricing = pricing.stack().sort_values()
sorted_pricing.head(20)

Region                      
us-west4                 E2     12.082960
europe-southwest1        T2D    13.496240
northamerica-northeast2  N2     14.033520
us-east5                 C2     14.272960
europe-west9             T2D    15.616160
us-east5                 T2D    15.832240
us-west1                 C2     16.451280
us-east1                 C2     16.451280
us-central1              C2     16.451280
europe-central2          E2     16.679040
me-west1                 C2     16.901544
                         C3     16.901544
asia-east2               E2     17.029440
europe-west12            N2D    17.362320
asia-south1              C2D    17.607600
europe-west8             C2     17.823446
europe-west9             C2     17.823446
                         C3     17.823446
europe-west8             C3     17.823446
europe-west12            C3     17.847098
dtype: float64

In [2]:
gpu.sort_values()

Region
europe-west2                50.98393
asia-northeast1             50.98393
europe-west6                80.30000
asia-northeast2             80.30000
us-east4                    80.30000
us-east5                    80.30000
asia-southeast1             80.30000
asia-southeast2             87.52700
europe-west8                93.14800
europe-west9                93.14800
us-south1                   94.75400
europe-southwest1           94.75400
asia-south2                 96.36000
us-east1                   102.20000
us-central1                102.20000
us-west1                   102.20000
asia-east1                 102.20000
europe-west4               102.20000
europe-west1               102.20000
europe-west12              103.58700
asia-northeast3            108.04000
us-west4                   108.04000
me-west1                   112.42000
northamerica-northeast1    112.42000
australia-southeast2       114.02600
southamerica-west1         114.82900
europe-west3               119.

In [ ]:
custom=df1.loc[:, ~df1.columns.str.contains("PREDEFINED|VMIMAGE|LOCAL-SSD|PUBLIC-IP-ADDRESS|GPU")& ~df1.columns.str.startswith("CUSTOM")]
df2 = df[df.index.isin(regions)]
predefined_vm_image_df=df2.loc[:,df2.columns.str.contains("VMIMAGE")]
#predefined_vm_image_df=predefined_vm_image_df.dropna(axis='columns')
predefined_vm_image_df

#low priority
#get cpus and ram for each machine type here manually

,E2-CUSTOM-VM-CORE,E2-CUSTOM-VM-RAM,F1-MICRO,G1-SMALL,N1-CUSTOM-VM-CORE,N1-CUSTOM-VM-RAM,N2-CUSTOM-VM-CORE,N2-CUSTOM-VM-RAM,N2D-CUSTOM-VM-CORE,N2D-CUSTOM-VM-RAM
Region,,,,,,,,,,
asia-east1,0.007955,0.001066,0.003900,0.007700,0.005840,0.000783,0.006199,0.000831,0.007680,0.001030
asia-east2,0.003968,0.000532,0.004900,0.009800,0.004433,0.000597,0.009630,0.001291,0.008308,0.001113
asia-northeast1,0.008408,0.001122,0.005000,0.010000,0.007930,0.001060,0.010149,0.001354,0.005534,0.000739
asia-northeast2,0.008408,0.001122,0.005000,0.010000,0.007950,0.001060,0.009280,0.001244,0.008964,0.001207
asia-northeast3,0.007567,0.001010,0.005000,0.010000,0.006490,0.000866,0.010821,0.001450,0.008964,0.001207
asia-south1,0.007074,0.000948,0.004200,0.008400,0.006473,0.000872,0.006742,0.001107,0.004843,0.000649
asia-south2,0.007860,0.001053,0.004200,0.008400,0.008376,0.001128,0.004012,0.000540,0.008376,0.001128
asia-southeast1,0.008476,0.001136,0.004000,0.008100,0.005890,0.000789,0.007776,0.001042,0.009000,0.001207
asia-southeast2,0.009165,0.001179,0.004360,0.008829,0.005781,0.000775,0.010789,0.001453,0.008502,0.001139


In [ ]:
df2 = df[df.index.isin(regions)]
predefined_vm_image_df=df2.loc[:,df2.columns.str.contains("VMIMAGE")]
#predefined_vm_image_df=predefined_vm_image_df.dropna(axis='columns')
predefined_vm_image_df

#low priority
#get cpus and ram for each machine type here manually

,VMIMAGE-E2-HIGHCPU-2,VMIMAGE-E2-HIGHCPU-4,VMIMAGE-E2-HIGHCPU-8,VMIMAGE-E2-HIGHCPU-16,VMIMAGE-E2-HIGHMEM-2,VMIMAGE-E2-HIGHMEM-4,VMIMAGE-E2-HIGHMEM-8,VMIMAGE-E2-HIGHMEM-16,VMIMAGE-E2-MEDIUM,VMIMAGE-E2-MICRO,...,VMIMAGE-N1-STANDARD-2,VMIMAGE-N1-STANDARD-4,VMIMAGE-N1-STANDARD-8,VMIMAGE-N1-STANDARD-16,VMIMAGE-N1-STANDARD-32,VMIMAGE-N1-STANDARD-64,VMIMAGE-N1-STANDARD-96,VMIMAGE-N1-ULTRAMEM-40,VMIMAGE-N1-ULTRAMEM-80,VMIMAGE-N1-ULTRAMEM-160
Region,,,,,,,,,,,,,,,,,,,,,
asia-east1,0.017180,0.034370,0.068740,0.137470,0.031400,0.062800,0.125610,0.251220,0.011640,0.002910,...,0.022,0.044,0.088,0.176,0.352,0.704,1.056,0,0,0
asia-east2,0.020770,0.041530,0.083060,0.166130,0.037950,0.075890,0.151780,0.303560,0.014060,0.003520,...,0.028,0.056,0.1121,0.2242,0.4484,0.8967,1.34505,0,0,0
asia-northeast1,0.019060,0.038120,0.076240,0.152470,0.034760,0.069530,0.139050,0.278110,0.012890,0.003220,...,0.0265,0.053,0.106,0.212,0.424,0.848,1.272,1.6682,3.3363,6.6726
asia-northeast2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012890,0.003220,...,0.0265,0.053,0.106,0.212,0.424,0.848,1.272,1.6682,3.3363,6.6726
asia-northeast3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012890,0.003220,...,0.0265,0.053,0.106,0.212,0.424,0.848,1.272,1.6682,3.3363,6.6726
asia-south1,0.017830,0.035650,0.071300,0.142610,0.032570,0.065140,0.130280,0.260570,0.012070,0.003020,...,0.02405,0.04811,0.09622,0.19243,0.38486,0.76973,0.96,0,0,0
asia-south2,0.017830,0.035650,0.071300,0.142610,0.032570,0.065140,0.130280,0.260570,0.012070,0.003020,...,0.02405,0.04811,0.09622,0.19243,0.38486,0.76973,0.96,0,0,0
asia-southeast1,0.018310,0.036620,0.073240,0.146470,0.033450,0.066910,0.133820,0.267630,0.012400,0.003100,...,0.0236,0.0472,0.0944,0.1888,0.3776,0.7552,1.1328,1.4808,2.9616,5.9232
asia-southeast2,0.019958,0.039916,0.079832,0.159652,0.036460,0.072932,0.145864,0.291717,0.013516,0.003379,...,0.025724,0.051448,0.102896,0.205792,0.411584,0.823168,1.234752,1.614072,3.228144,6.456288
